# Data Extraction - Azure OpenAI GPT-4o with Vision

This sample demonstrates how to extract structured data from any document using Azure OpenAI's GPT-4o model with vision capabilities.

![Data Extraction](../../../images/extraction-openai.png)

This is achieved by the following process:

- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes the specific extraction instruction for the type of document, and each document page as a base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from the document page images using Azure OpenAI's GPT-4o model.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **pdf2image** for converting a PDF file into a set of images per page.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.comparison** to compare the output of the extraction process with expected results.
- **modules.document_processing_result** to store the results of the extraction process as a file.
- **modules.openai_confidence** to calculate the confidence of the classification process based on the `logprobs` response from the API request.
- **modules.vehicle_insurance_policy** to provide the expected structured output JSON schema for vehicle insurance policy documents.
- **modules.stopwatch** to measure the end-to-end execution time for the classification process.

In [1]:
import sys
sys.path.append('../../') # Import local modules

from IPython.display import display, Markdown
import os
import pandas as pd
from dotenv import dotenv_values
from pdf2image import convert_from_bytes
import base64
import io
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

from modules.app_settings import AppSettings
from modules.comparison import extraction_comparison
from modules.document_processing_result import DataExtractionResult
from modules.openai_confidence import evaluate_confidence
from modules.vehicle_insurance_policy import VehicleInsurancePolicy, VehicleInsuranceCostDetails, VehicleInsuranceRenewalDetails, VehicleInsurancePersonDetails, VehicleInsuranceVehicleDetails, VehicleInsuranceExcessDetails, VehicleInsurancePolicyEvaluator
from modules.stopwatch import Stopwatch

### Configure the Azure services

To use Azure OpenAI, the SDK is used to create a client instance using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-08-01-preview" # Requires the latest API version for structured outputs.
)

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on each page of the [Vehicle Insurance Policy](../../assets/VehicleInsurancePolicy.pdf).

The expected output has been defined by a human evaluating the document.

In [3]:
pdf_path = f"{working_dir}/samples/assets/"
pdf_file_name = "VehicleInsurancePolicy.pdf"
fname = f"{pdf_path}{pdf_file_name}"

expected = VehicleInsurancePolicy(
    policy_number='GB20246717948',
    cost=VehicleInsuranceCostDetails(
        annual_total=532.19,
        payable_by_date='2024-06-13'
    ),
    renewal=VehicleInsuranceRenewalDetails(
        renewal_notification_date='2025-05-12',
        last_date_to_renew='2025-05-26'
    ),
    effective_from='2024-06-03',
    effective_to='2025-06-02',
    last_date_to_cancel='2024-06-17',
    policyholder=VehicleInsurancePersonDetails(
        first_name='Joe',
        last_name='Bloggs',
        date_of_birth='1990-01-05',
        address='73 Regal Way, LEEDS, West Yorkshire, LS1 5AB',
        email_address='Joe.Bloggs@me.com',
        total_years_of_residence_in_uk=34,
        driving_license_number='BLOGGS901050JJ1AB'
    ),
    vehicle=VehicleInsuranceVehicleDetails(
        registration_number='VS24DMC',
        make='Hyundai',
        model='IONIQ 5 Premium 73 kWh RWD',
        year=2024,
        value=40000
    ),
    accident_excess=VehicleInsuranceExcessDetails(
        compulsory=250,
        voluntary=250,
        unapproved_repair_penalty=250
    ),
    fire_and_theft_excess=VehicleInsuranceExcessDetails(
        compulsory=250,
        voluntary=250,
        unapproved_repair_penalty=250
    ),
)

insurance_policy_evaluator = VehicleInsurancePolicyEvaluator(expected)

## Extract data from the document

The following code block executes the data extraction process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use pdf2image to convert the document's pages into images per page as base64 strings.
3. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the images.

In [4]:
# Prepare the user content for the OpenAI API including any specific details for processing this type of document and the document page images.
user_content = []
user_content.append({
    "type": "text",
    "text": f"""Extract the data from this insurance policy. 
    - If a value is not present, provide null.
    - Some values must be inferred based on the rules defined in the policy.
    - Dates should be in the format YYYY-MM-DD."""
})

with Stopwatch() as image_stopwatch:
    document_bytes = open(fname, "rb").read()
    page_images = convert_from_bytes(document_bytes)
    for page_image in page_images:
        byteIO = io.BytesIO()
        page_image.save(byteIO, format='PNG')
        base64_data = base64.b64encode(byteIO.getvalue()).decode('utf-8')
        
        user_content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_data}"
            }
        })

with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that extracts data from documents.",
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        response_format=VehicleInsurancePolicy,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1,
        logprobs=True # Enabled to determine the confidence of the response.
    )

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction based on the log probability of the output generated by Azure OpenAI's GPT-4o model.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [5]:
# Gets the parsed VehicleInsurancePolicy object from the completion response.
insurance_policy = completion.choices[0].message.parsed

# Determines the accuracy of the extracted data against the expected values.
accuracy = insurance_policy_evaluator.evaluate(insurance_policy)

# Determines the confidence of the extracted data using the log probabilities of the completion response.
confidence = evaluate_confidence(insurance_policy.to_dict(), completion.choices[0])

# Gets the total execution time of the data extraction process.
total_elapsed = image_stopwatch.elapsed + oai_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

# Save the output of the data extraction result.
extraction_result = DataExtractionResult(insurance_policy.to_dict(), confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

with open(f"{working_dir}/samples/extraction/vision-based/openai.{pdf_file_name}.json", "w") as f:
    f.write(extraction_result.to_json(indent=4))
    
# Display the outputs of the data extraction process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Image Pre-processing Execution Time": f"{image_stopwatch.elapsed:.2f} seconds",
        "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(Markdown(df.to_markdown(index=False, tablefmt='unsafehtml')))
display(Markdown(extraction_comparison(expected.to_dict(), insurance_policy.to_dict(), confidence)))

<table>
<thead>
<tr><th>Accuracy  </th><th>Confidence  </th><th>Execution Time  </th><th>Image Pre-processing Execution Time  </th><th>OpenAI Execution Time  </th><th style="text-align: right;">  Prompt Tokens</th><th style="text-align: right;">  Completion Tokens</th></tr>
</thead>
<tbody>
<tr><td>98.57%    </td><td>97.50%      </td><td>26.96 seconds   </td><td>7.37 seconds                         </td><td>19.59 seconds          </td><td style="text-align: right;">           9851</td><td style="text-align: right;">                266</td></tr>
</tbody>
</table>

<table>
<thead>
<tr><th>Field                                          </th><th>Expected                                    </th><th>Extracted                                                                     </th><th style="text-align: right;">  Confidence</th></tr>
</thead>
<tbody>
<tr><td>policy_number                                  </td><td>GB20246717948                               </td><td><span style='color: green'>GB20246717948</span>                               </td><td style="text-align: right;">    0.999699</td></tr>
<tr><td>cost_annual_total                              </td><td>532.19                                      </td><td><span style='color: green'>532.19</span>                                      </td><td style="text-align: right;">    0.999948</td></tr>
<tr><td>cost_payable_by_date                           </td><td>2024-06-13                                  </td><td><span style='color: green'>2024-06-13</span>                                  </td><td style="text-align: right;">    0.999108</td></tr>
<tr><td>renewal_renewal_notification_date              </td><td>2025-05-12                                  </td><td><span style='color: green'>2025-05-12</span>                                  </td><td style="text-align: right;">    0.964622</td></tr>
<tr><td>renewal_last_date_to_renew                     </td><td>2025-05-26                                  </td><td><span style='color: green'>2025-05-26</span>                                  </td><td style="text-align: right;">    0.97325 </td></tr>
<tr><td>effective_from                                 </td><td>2024-06-03                                  </td><td><span style='color: green'>2024-06-03</span>                                  </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>effective_to                                   </td><td>2025-06-02                                  </td><td><span style='color: green'>2025-06-02</span>                                  </td><td style="text-align: right;">    0.999996</td></tr>
<tr><td>last_date_to_cancel                            </td><td>2024-06-17                                  </td><td><span style='color: green'>2024-06-17</span>                                  </td><td style="text-align: right;">    0.993498</td></tr>
<tr><td>policyholder_first_name                        </td><td>Joe                                         </td><td><span style='color: green'>Joe</span>                                         </td><td style="text-align: right;">    0.999776</td></tr>
<tr><td>policyholder_last_name                         </td><td>Bloggs                                      </td><td><span style='color: green'>Bloggs</span>                                      </td><td style="text-align: right;">    1       </td></tr>
<tr><td>policyholder_date_of_birth                     </td><td>1990-01-05                                  </td><td><span style='color: green'>1990-01-05</span>                                  </td><td style="text-align: right;">    0.990891</td></tr>
<tr><td>policyholder_address                           </td><td>73 Regal Way, LEEDS, West Yorkshire, LS1 5AB</td><td><span style='color: green'>73 Regal Way, LEEDS, West Yorkshire, LS1 5AB</span></td><td style="text-align: right;">    0.968838</td></tr>
<tr><td>policyholder_email_address                     </td><td>Joe.Bloggs@me.com                           </td><td><span style='color: green'>joe.Bloggs@me.com</span>                           </td><td style="text-align: right;">    0.900354</td></tr>
<tr><td>policyholder_total_years_of_residence_in_uk    </td><td>34                                          </td><td><span style='color: green'>34</span>                                          </td><td style="text-align: right;">    0.824488</td></tr>
<tr><td>policyholder_driving_license_number            </td><td>BLOGGS901050JJ1AB                           </td><td><span style='color: red'>BLOGGS901050J1AB</span>                              </td><td style="text-align: right;">    0.964675</td></tr>
<tr><td>vehicle_registration_number                    </td><td>VS24DMC                                     </td><td><span style='color: green'>VS24DMC</span>                                     </td><td style="text-align: right;">    0.999996</td></tr>
<tr><td>vehicle_make                                   </td><td>Hyundai                                     </td><td><span style='color: green'>Hyundai</span>                                     </td><td style="text-align: right;">    0.999544</td></tr>
<tr><td>vehicle_model                                  </td><td>IONIQ 5 Premium 73 kWh RWD                  </td><td><span style='color: green'>IONIQ 5 Premium 73 kWh RWD</span>                  </td><td style="text-align: right;">    0.999196</td></tr>
<tr><td>vehicle_year                                   </td><td>2024                                        </td><td><span style='color: green'>2024</span>                                        </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>vehicle_value                                  </td><td>40000.0                                     </td><td><span style='color: green'>40000.0</span>                                     </td><td style="text-align: right;">    0.823848</td></tr>
<tr><td>accident_excess_compulsory                     </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999993</td></tr>
<tr><td>accident_excess_voluntary                      </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999998</td></tr>
<tr><td>accident_excess_unapproved_repair_penalty      </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.948777</td></tr>
<tr><td>fire_and_theft_excess_compulsory               </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>fire_and_theft_excess_voluntary                </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999919</td></tr>
<tr><td>fire_and_theft_excess_unapproved_repair_penalty</td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999022</td></tr>
</tbody>
</table>